In [0]:
import numpy as np
from matplotlib import pyplot as plt


import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, Lambda
from keras.layers.core import Lambda, RepeatVector, Reshape
from tensorflow.keras.layers  import Conv2D, Conv2DTranspose
from tensorflow.keras.layers  import MaxPooling2D, GlobalMaxPool2D
from tensorflow.keras.layers  import concatenate, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import Adam

In [0]:
#For loading our files from our google drive account:
from google.colab import drive
drive.mount('/content/gdrive')

Definimos la U-Net
![texto alternativo](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png)

In [0]:
#UNet code:
# 1) Since each block consicts on two convolutions:

def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x


# 2) We create a function to define the UNET Model
def get_unet(inputs, n_filters=32, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(inputs, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [0]:
gpus = tf.config.experimental.list_physical_devices("GPU")
print("Num GPUs Available: ", len(gpus))

In [0]:
if len(gpus) >= 1:
  config = tf.compat.v1.ConfigProto()
  config.gpu_options.allow_growth = True
  tf.compat.v1.Session(config = config)

In [0]:
#In this line we willspecify the UNet what it should expect

# Specifying the input shape
input_shape = Input((512, 512, 3), dtype = 'float32')

model = get_unet(input_shape, n_filters=32, dropout=0.1, batchnorm=True)
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

In [0]:
#Loading our arrays saved as npy
#In order for this to work you should write your own corresponding paths

X_train = np.load('X_train_path.npy')
y_train_MASK= np.load('y_train_MASK_path.npy')

X_test = np.load('X_test_PATH.npy')
y_test_MASK = np.load('/y_test_MASK_path.npy')

In [0]:
#Just for checking that everything was uploaded without any problems
#The dimensions of X_train = y_train_MASK and the dimensions of X_test = y_test_MASK

print(X_train.shape, y_train_MASK.shape, X_test.shape, y_test_MASK.shape)

In [0]:
#For training the model.
# We use some callbacks in order to avoid overfitting y underfitting, and additionally to save our best model.
#Change the paths for each mask.

modelCallbacks = [
    EarlyStopping(monitor = 'val_loss', patience=10, verbose=1),
    ReduceLROnPlateau(monitor = 'val_loss', factor=0.2, patience=6 , min_lr=0.00001, verbose=1),
    ModelCheckpoint('path/model-MASK.h5', verbose=1, save_best_only=True, save_weights_only=True),
    TensorBoard(log_dir = 'path/MASK/logs/MASK_1')
    ]


results = model.fit(X_train, y_train_MASK, batch_size= 16, epochs=100, verbose = 2, callbacks=modelCallbacks, validation_data=(X_test, y_test_MASK))

In [0]:
#Plot the learning curve and mark the best model:

plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

In [0]:
# Now load weights from the best saved model:
# Note that the exact same path that was saved in the callback modelcheckpoint must be written.

model.load_weights('path/model-MASK.h5')

# Evaluate on test set.
model.evaluate(X_test, y_test_MASK, verbose=1)

In [0]:
#Check the max value from a random pred train, it is usedfull for choosing the threshold in the next step

print(np.amin(preds_test[16]),np.amax(preds_test[16]))

In [0]:
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

In [0]:
# Used to plot the original image, the original mask, the UNet predicted mask and the UNet predicted binary mask
def plot_sample(X, y, preds, binary_preds, ix=None):
  plt.imshow(X[ix],cmap='gray')
  plt.xticks([]), plt.yticks([])  
  plt.xlabel('Retina image')
  plt.show()

  plt.imshow(y[ix].squeeze(), cmap='gray',vmin=0,vmax=1)
  plt.xticks([]), plt.yticks([])  
  plt.xlabel('Original mask')
  plt.show()

  plt.imshow(preds[ix].squeeze(), cmap='gray')
  plt.xticks([]), plt.yticks([])  
  plt.xlabel('Mask prediction')
  plt.show()

  plt.imshow(binary_preds[ix].squeeze(), cmap='gray',vmin=0,vmax=1)
  plt.xticks([]), plt.yticks([])  
  plt.xlabel('Binary mask prediction')
  plt.show()

In [0]:
# Now lets plot some samples from the train set:
# Bear in mind that ix should be a number

plot_sample(X_train, y_train_MASK, preds_train, preds_train_t, ix= X)

In [0]:
# Now lets plot some samples from the test set:
# Bear in mind that ix should be a number

plot_sample(X_train, y_test_MASK, preds_test, preds_test_t, ix= X)